In [ ]:
# YENİ VERİ TOPLAMA VE BİRLEŞTİRME

# Önceki veriyi yükle
df_final_existing = pd.read_csv("Somali_News_Classification_Dataset_Final.csv")

# Yeni toplanacak veriler için boş liste
new_data = []

print("🚀 Yeni Spor ve Sağlık verileri toplanıyor...")

# Goobjoog Sports verisi çek
print("--- Goobjoog Sports taranıyor ---")
new_data.extend(scrape_goobjoog_category("https://goobjoog.com/qayb/cayaaraha/", "Sports", max_pages=20))

# Goobjoog Health verisi çek
print("--- Goobjoog Health taranıyor ---")
new_data.extend(scrape_goobjoog_category("https://goobjoog.com/qayb/saynis-tiknooloji/caafimaadka/", "Health", max_pages=20))

# VOA Somali Sports RSS'ini keşfet ve çek
print("--- VOA Somali Sports RSS taranıyor ---")
voa_sports_rss_url = discover_voa_rss("Ciyaaraha")
if voa_sports_rss_url:
    new_data.extend(fetch_rss_items(voa_sports_rss_url, "VOA Somali", fixed_label="Sports", max_items=200))
    print(f"VOA Sports RSS'ten {len(new_data) - (df_final_existing.shape[0] if 'url' in df_final_existing.columns else 0)} yeni haber eklendi.")
else:
    print("VOA Sports RSS linki bulunamadı.")

# VOA Somali Health RSS'ini keşfet ve çek (eğer varsa)
print("--- VOA Somali Health RSS taranıyor ---")
voa_health_rss_url = discover_voa_rss("Caafimaadka") # 'Caafimaadka' sağlık anlamına gelir
if voa_health_rss_url:
    current_new_data_len = len(new_data)
    new_data.extend(fetch_rss_items(voa_health_rss_url, "VOA Somali", fixed_label="Health", max_items=200))
    print(f"VOA Health RSS'ten {len(new_data) - current_new_data_len} yeni haber eklendi.")
else:
    print("VOA Health RSS linki bulunamadı.")

# Yeni veriyi DataFrame'e dönüştür
df_new = pd.DataFrame(new_data)

# Gereksiz sütunları kaldırıp sadece 'text' ve 'label' tut
df_new = df_new[['text', 'label']].copy()

# Mevcut ve yeni veriyi birleştir
df_combined = pd.concat([df_final_existing, df_new], ignore_index=True)

# Duplike verileri temizle (özellikle 'text' sütununa göre)
df_combined.drop_duplicates(subset=['text'], inplace=True)

# Veriyi karıştır
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Son durumu göster
print("\n--- Güncellenmiş Final Raporu ---")
print(df_combined['label'].value_counts())
print(f"Toplam benzersiz veri sayısı: {len(df_combined)}")

# Yeni final dosyasını kaydet
df_combined.to_csv("Somali_News_Classification_Dataset_Final_Updated.csv", index=False)
print("✅ Güncellenmiş dataset 'Somali_News_Classification_Dataset_Final_Updated.csv' olarak kaydedildi.")


FileNotFoundError: [Errno 2] No such file or directory: 'Somali_News_Classification_Dataset_Final.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# Hedef site: Caasimada (Örnek olarak, yapısı çok düzenlidir)
# Bu URL yapıları örnektir, siteye girip kontrol etmek gerekir.
# Eğer linkler değişmişse 'categories' kısmını güncellemelisin.

categories = {
    # Kategori Adı : (Link Kalıbı, Sayfa Sayısı Limiti)
    "Politics": ("https://www.caasimada.net/category/wararka/page/", 250),
    "World": ("https://www.caasimada.net/category/caalamka/page/", 250),
    "Sports": ("https://www.caasimada.net/category/ciyaaraha/page/", 250),
    "Opinion": ("https://www.caasimada.net/category/aragtida-dadweynaha/page/", 250)
}

data = []

print("Veri toplama işlemi başlıyor... Hedef: ~10.000 satır")

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for category, (base_url, max_pages) in categories.items():
    print(f"\n--- {category} kategorisi taranıyor ---")

    for page_num in range(1, max_pages + 1):
        full_url = f"{base_url}{page_num}/"

        try:
            response = requests.get(full_url, headers=headers, timeout=10)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # Caasimada sitesine özel başlık yakalama (Genelde h1, h2 veya h3 olur)
                # Sitede sağ tıklayıp İncele diyerek class ismini bulmak gerekebilir.
                # Genelde linklerin içindeki text başlık olur.
                articles = soup.find_all('h2', class_='post-title')

                # Eğer h2 ile bulamazsa alternatif h3 deneyelim (Yedek plan)
                if not articles:
                    articles = soup.find_all('h3')

                page_count = 0
                for article in articles:
                    title = article.get_text().strip()
                    if len(title) > 15: # Çok kısa anlamsız başlıkları atla
                        data.append({
                            "text": title,
                            "label": category,
                            "source": "Caasimada"
                        })
                        page_count += 1

                print(f"Sayfa {page_num}: {page_count} haber eklendi. (Toplam Veri: {len(data)})")

            else:
                print(f"Sayfa {page_num} açılamadı. Kod: {response.status_code}")

            # Site bizi robot sanıp engellemesin diye her sayfada biraz bekleyelim
            time.sleep(random.uniform(0.5, 1.5))

        except Exception as e:
            print(f"Hata oluştu (Sayfa {page_num}): {e}")
            continue

# Veriyi tabloya çevir
df = pd.DataFrame(data)

# Veriyi kaydet
df.to_csv("somali_news_dataset_10k.csv", index=False)
print(f"\nİŞLEM TAMAMLANDI! Toplam {len(df)} adet veri toplandı.")
print(df.head())

Veri toplama işlemi başlıyor... Hedef: ~10.000 satır

--- Politics kategorisi taranıyor ---
Sayfa 1: 26 haber eklendi. (Toplam Veri: 26)
Sayfa 2: 21 haber eklendi. (Toplam Veri: 47)
Sayfa 3: 21 haber eklendi. (Toplam Veri: 68)
Sayfa 4: 21 haber eklendi. (Toplam Veri: 89)
Sayfa 5: 21 haber eklendi. (Toplam Veri: 110)
Sayfa 6: 21 haber eklendi. (Toplam Veri: 131)
Sayfa 7: 21 haber eklendi. (Toplam Veri: 152)
Sayfa 8: 21 haber eklendi. (Toplam Veri: 173)
Sayfa 9: 21 haber eklendi. (Toplam Veri: 194)
Sayfa 10: 21 haber eklendi. (Toplam Veri: 215)
Sayfa 11: 21 haber eklendi. (Toplam Veri: 236)
Sayfa 12: 21 haber eklendi. (Toplam Veri: 257)
Sayfa 13: 21 haber eklendi. (Toplam Veri: 278)
Sayfa 14: 21 haber eklendi. (Toplam Veri: 299)
Sayfa 15: 21 haber eklendi. (Toplam Veri: 320)
Sayfa 16: 21 haber eklendi. (Toplam Veri: 341)
Sayfa 17: 21 haber eklendi. (Toplam Veri: 362)
Sayfa 18: 21 haber eklendi. (Toplam Veri: 383)
Sayfa 19: 21 haber eklendi. (Toplam Veri: 404)
Sayfa 20: 21 haber eklendi. 

KeyboardInterrupt: 

In [ ]:
# 1. Mevcut veriyi hemen CSV'ye döküyoruz
df_part1 = pd.DataFrame(data)
df_part1.to_csv("somali_news_part1.csv", index=False)
print(f"Harika! {len(df_part1)} satır veri 'somali_news_part1.csv' olarak kaydedildi.")

Harika! 10510 satır veri 'somali_news_part1.csv' olarak kaydedildi.


In [ ]:
print(f"There are {df_part1['label'].nunique()} unique labels in the DataFrame.")
print("The unique labels are:")
print(df_part1['label'].unique())

There are 2 unique labels in the DataFrame.
The unique labels are:
['Politics' 'World']


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from urllib.parse import urljoin

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

categories = {
    # Goobjoog
    "Sports":  {"base": "https://goobjoog.com/qayb/cayaaraha/", "max_pages": 20, "source": "Goobjoog"},
    "Health":  {"base": "https://goobjoog.com/qayb/saynis-tiknooloji/caafimaadka/", "max_pages": 20, "source": "Goobjoog"},

    # Jowhar
    "Sports_Jowhar": {"base": "https://jowhar.com/category/sport/", "max_pages": 20, "source": "Jowhar"},
}

data = []

def build_page_url(base, page):
    base = base.rstrip("/") + "/"
    if page == 1:
        return base
    return urljoin(base, f"page/{page}/")

for key, cfg in categories.items():
    label = "Sports" if "Sports" in key else key
    base = cfg["base"]
    max_pages = cfg["max_pages"]
    source = cfg["source"]

    print(f"\n--- {key} ({label}) taranıyor ---")
    for page in range(1, max_pages + 1):
        url = build_page_url(base, page)

        try:
            r = requests.get(url, headers=headers, timeout=20, allow_redirects=True)
            if r.status_code != 200:
                print(f"Sayfa {page}: HTTP {r.status_code} -> {url}")
                break

            soup = BeautifulSoup(r.text, "html.parser")

            # En güvenlisi: başlık linklerini almak
            links = soup.select("h2 a, h3 a")

            count = 0
            for a in links:
                title = a.get_text(strip=True)
                href = a.get("href")

                # Çok kısa/boş başlıkları ele
                if not title or len(title) < 15:
                    continue

                data.append({
                    "text": title,
                    "label": label,
                    "url": href,
                    "source": source
                })
                count += 1

            print(f"Sayfa {page}: {count} haber alındı.")

            # Sayfa 200 dönse bile içerik yoksa dur
            if count == 0:
                print("Başlık bulunamadı; kategori bitmiş veya HTML yapısı farklı.")
                break

            time.sleep(random.uniform(0.8, 1.6))

        except requests.exceptions.RequestException as e:
            print(f"Hata (requests): {e}")
            continue

if data:
    df = pd.DataFrame(data).drop_duplicates(subset=["text", "url"])
    df.to_csv("somali_news_fixed.csv", index=False, encoding="utf-8-sig")
    print(f"\nBAŞARILI: {len(df)} kayıt yazıldı -> somali_news_fixed.csv")
else:
    print("\nVeri gelmedi. (1) Site bot engeli (2) HTML selector uyumsuzluğu (3) erişim problemi olabilir.")



--- Sports (Sports) taranıyor ---
Sayfa 1: 10 haber alındı.
Sayfa 2: 10 haber alındı.
Sayfa 3: 10 haber alındı.
Sayfa 4: 10 haber alındı.
Sayfa 5: 10 haber alındı.
Sayfa 6: 10 haber alındı.
Sayfa 7: 10 haber alındı.
Sayfa 8: 10 haber alındı.
Sayfa 9: 10 haber alındı.
Sayfa 10: 10 haber alındı.
Sayfa 11: 10 haber alındı.
Sayfa 12: 10 haber alındı.
Sayfa 13: 10 haber alındı.
Sayfa 14: 10 haber alındı.
Sayfa 15: 10 haber alındı.
Sayfa 16: 10 haber alındı.
Sayfa 17: 10 haber alındı.
Sayfa 18: 10 haber alındı.
Sayfa 19: 10 haber alındı.
Sayfa 20: 10 haber alındı.

--- Health (Health) taranıyor ---
Sayfa 1: 10 haber alındı.
Sayfa 2: 10 haber alındı.
Sayfa 3: 10 haber alındı.
Sayfa 4: 10 haber alındı.
Sayfa 5: 10 haber alındı.
Sayfa 6: 10 haber alındı.
Sayfa 7: 10 haber alındı.
Sayfa 8: 10 haber alındı.
Sayfa 9: 10 haber alındı.
Sayfa 10: 10 haber alındı.
Sayfa 11: 10 haber alındı.
Sayfa 12: 10 haber alındı.
Sayfa 13: 10 haber alındı.
Sayfa 14: 10 haber alındı.
Sayfa 15: 10 haber alındı.
Say

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import time
# import random

# # YENİ HEDEF LİSTESİ: Daha yerel ve erişilebilir kaynaklar
# # Bu siteler genelde 'category/ciyaaraha' yapısını kullanır.
# targets = [
#     {
#         "name": "Radio Risaala (Spor)",
#         "base_url": "https://radiorisaala.com/category/ciyaaraha/page/",
#         "label": "Sports",
#         "max_pages": 50
#     },
#     {
#         "name": "Radio Risaala (Sağlık)",
#         "base_url": "https://radiorisaala.com/category/caafimaadka/page/",
#         "label": "Health",
#         "max_pages": 30
#     },
#     {
#         "name": "Radio Dalsan (Spor)",
#         "base_url": "https://www.radiodalsan.com/category/ciyaaraha/page/",
#         "label": "Sports",
#         "max_pages": 50
#     },
#     {
#         "name": "Muqdisho Online (Spor)",
#         "base_url": "https://muqdisho.online/category/ciyaaraha/page/",
#         "label": "Sports",
#         "max_pages": 40
#     }
# ]

# data_rescue = []
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# print("🚀 C PLANI DEVREDE: Spor ve Sağlık Verisi Aranıyor...")

# for target in targets:
#     print(f"\n--- {target['name']} Taranıyor ---")

#     # Önce ilk sayfayı kontrol et, 404 verirse hiç uğraşma
#     test_url = f"{target['base_url']}1/"
#     try:
#         test_response = requests.get(test_url, headers=headers, timeout=15)
#         if test_response.status_code != 200:
#             print(f"❌ {target['name']} erişilemedi (Kod: {test_response.status_code}). Geçiliyor...")
#             continue
#     except:
#         print(f"❌ {target['name']} bağlantı hatası. Geçiliyor...")
#         continue

#     # Site çalışıyorsa veriyi çek
#     success_count = 0
#     for page_num in range(1, target['max_pages'] + 1):
#         full_url = f"{target['base_url']}{page_num}/"

#         try:
#             response = requests.get(full_url, headers=headers, timeout=10)
#             if response.status_code == 200:
#                 soup = BeautifulSoup(response.content, 'html.parser')

#                 # Bu sitelerin hepsi WordPress, genelde h2 veya h3 başlık kullanır
#                 articles = soup.find_all('h2')
#                 if len(articles) < 3: articles = soup.find_all('h3') # Yedek
#                 if len(articles) < 3: articles = soup.find_all('h1') # Yedek

#                 page_news_count = 0
#                 for article in articles:
#                     title = article.get_text().strip()
#                     if len(title) > 20:
#                         data_rescue.append({
#                             "text": title,
#                             "label": target['label'],
#                             "source": target['name']
#                         })
#                         page_news_count += 1

#                 success_count += page_news_count
#                 print(f"Sayfa {page_num}: {page_news_count} haber eklendi.")

#                 if page_news_count == 0:
#                     print("Sayfa boş, döngü kırılıyor.")
#                     break
#             else:
#                 break # Sayfa bitti

#             time.sleep(random.uniform(0.5, 1.0))

#         except Exception as e:
#             print(f"Hata: {e}")
#             break

#     print(f"✅ {target['name']} Tamamlandı: Toplam {success_count} veri.")

# # Sonuçları Kaydet
# if len(data_rescue) > 0:
#     df_rescue = pd.DataFrame(data_rescue)
#     df_rescue.to_csv("somali_news_sports_health_C_PLAN.csv", index=False)
#     print(f"\n🎉 TEBRİKLER! Toplam {len(df_rescue)} adet yeni veri kurtarıldı.")
# else:
#     print("\n⚠️ Hala veri yok. Bu durumda 'Manuel Ekleme' yöntemine geçeceğiz.")

🚀 C PLANI DEVREDE: Spor ve Sağlık Verisi Aranıyor...

--- Radio Risaala (Spor) Taranıyor ---
❌ Radio Risaala (Spor) erişilemedi (Kod: 404). Geçiliyor...

--- Radio Risaala (Sağlık) Taranıyor ---
❌ Radio Risaala (Sağlık) erişilemedi (Kod: 404). Geçiliyor...

--- Radio Dalsan (Spor) Taranıyor ---
Sayfa 1: 0 haber eklendi.
Sayfa boş, döngü kırılıyor.
✅ Radio Dalsan (Spor) Tamamlandı: Toplam 0 veri.

--- Muqdisho Online (Spor) Taranıyor ---
❌ Muqdisho Online (Spor) erişilemedi (Kod: 404). Geçiliyor...

⚠️ Hala veri yok. Bu durumda 'Manuel Ekleme' yöntemine geçeceğiz.


In [ ]:
import pandas as pd

# 1. Adım: Önceki büyük dosyanı yükle (Politika ve Dünya haberleri)
# Dosya adın 'somali_news_part1.csv' veya benzeriydi, onu buraya yaz.
df_politics = pd.read_csv("somali_news_part1.csv")

# 2. Adım: Şimdi bu yeni kodun ürettiği dosyayı yükle (Spor ve Sağlık)
df_sports_health = pd.read_csv("somali_news_fixed.csv")

# Sütun isimlerini eşitleyelim (Garanti olsun)
# Yeni kodunda sütun adı 'url', eskisinde 'source' veya farklı olabilir.
# Sadece 'text' ve 'label' bizim için kritik.
df_politics = df_politics[['text', 'label']]
df_sports_health = df_sports_health[['text', 'label']]

# 3. Adım: Hepsini birleştir
df_final = pd.concat([df_politics, df_sports_health], ignore_index=True)

# 4. Adım: Verileri karıştır (Shuffle) - Eğitim için şart!
df_final = df_final.sample(frac=1).reset_index(drop=True)

# 5. Adım: Kontrol et ve Kaydet
print("--- FİNAL RAPORU ---")
print(df_final['label'].value_counts()) # Hangi kategoriden kaç tane var?
print(f"Toplam Veri Sayısı: {len(df_final)}")

df_final.to_csv("Somali_News_Classification_Dataset_Final.csv", index=False)
print("✅ MUTLU SON! 'Somali_News_Classification_Dataset_Final.csv' dosyası hazır.")

FileNotFoundError: [Errno 2] No such file or directory: 'somali_news_part1.csv'

🚀 SON KURTARMA OPERASYONU BAŞLADI...

--- Laacib Sports Taranıyor ---
Sayfa 1 erişim hatası (Kod: 403)
Sayfa 2 erişim hatası (Kod: 403)
Sayfa 3 erişim hatası (Kod: 403)
Sayfa 4 erişim hatası (Kod: 403)
Sayfa 5 erişim hatası (Kod: 403)
Sayfa 6 erişim hatası (Kod: 403)
Sayfa 7 erişim hatası (Kod: 403)
Sayfa 8 erişim hatası (Kod: 403)
Sayfa 9 erişim hatası (Kod: 403)


KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import xml.etree.ElementTree as ET

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

TIMEOUT = 25

# -----------------------------
# 1) Labeling (keyword-based)
# -----------------------------
SPORTS_KW = [
    "ciyaar", "ciyaaraha", "kubad", "kubbad", "k/cag", "cagta", "football",
    "soccer", "premier", "la liga", "serie a", "bundesliga", "champions",
    "olympic", "olombik", "final", "goal", "gool", "tartan", "koob", "cup",
    "nba", "fifa", "caf", "uefa", "man city", "manchester", "arsenal",
    "barcelona", "real madrid", "liverpool", "milan", "inter"
]
HEALTH_KW = [
    "caafimaad", "caafimaadka", "cudur", "xanuun", "dhakhtar", "isbitaal",
    "tallaal", "vaccin", "vaccine", "covid", "malaria", "cholera", "dengue",
    "hargab", "daawo", "WHO", "unicef", "nafaqo", "nutrition", "hooyo",
    "uur", "caruur", "bukaan", "epidemic", "virus", "infection"
]

def classify_label(title: str) -> str:
    t = title.lower()
    s_score = sum(1 for k in SPORTS_KW if k in t)
    h_score = sum(1 for k in HEALTH_KW if k.lower() in t)
    if s_score == 0 and h_score == 0:
        return "Other"
    return "Sports" if s_score >= h_score else "Health"

# -----------------------------
# 2) RSS helpers
# -----------------------------
def fetch_rss_items(rss_url: str, source: str, fixed_label: str | None = None, max_items: int | None = None):
    """
    Pull RSS XML and return list of dicts.
    If fixed_label is None -> classify by keywords.
    """
    out = []
    r = requests.get(rss_url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
    r.raise_for_status()

    text = r.text.strip()
    # Quick sanity check: must look like XML
    if not (text.startswith("<?xml") or "<rss" in text or "<feed" in text):
        return out

    root = ET.fromstring(text)

    # RSS 2.0 items
    items = root.findall(".//item")
    if not items:
        # Atom fallback
        items = root.findall(".//{http://www.w3.org/2005/Atom}entry")

    for it in items[:max_items] if max_items else items:
        # RSS
        title = (it.findtext("title") or it.findtext("{http://www.w3.org/2005/Atom}title") or "").strip()
        link = (it.findtext("link") or "").strip()
        pub = (it.findtext("pubDate") or it.findtext("{http://www.w3.org/2005/Atom}updated") or "").strip()

        if not title or len(title) < 12:
            continue

        label = fixed_label if fixed_label else classify_label(title)
        if label == "Other":
            # İstersen bunu kaldırıp "Other"ları da saklayabilirsin
            continue

        out.append({
            "text": title,
            "label": label,
            "source": source,
            "url": link,
            "published": pub
        })

    return out

# -----------------------------
# 3) VOA: auto-discover Sports RSS from rssfeeds page
# -----------------------------
def discover_voa_rss(category_text: str = "Ciyaaraha"):
    """
    Tries to find the RSS subscribe link (Rukumo) for a category on VOA rssfeeds page.
    Returns URL or None.
    """
    rssfeeds_url = "https://www.voasomali.com/rssfeeds"
    r = requests.get(rssfeeds_url, headers=HEADERS, timeout=TIMEOUT)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # Find list item containing the category name, then find an <a> with href containing "/api/"
    # The page typically lists: "* CategoryName ... Rukumo"
    for li in soup.select("li"):
        txt = li.get_text(" ", strip=True)
        if category_text.lower() in txt.lower():
            a = li.find("a", href=True)
            # Some pages have multiple <a>; prioritize api links
            if a and "/api/" in a["href"]:
                return a["href"]

            # fallback: scan all links in li
            for aa in li.find_all("a", href=True):
                if "/api/" in aa["href"]:
                    return aa["href"]

    # Extra fallback: scan all links globally
    for aa in soup.find_all("a", href=True):
        if "/api/" in aa["href"] and category_text.lower() in aa.get_text(strip=True).lower():
            return aa["href"]

    return None

# -----------------------------
# 4) Goobjoog HTML scraper (correct URLs)
# -----------------------------
def scrape_goobjoog_category(base_url: str, label: str, max_pages: int = 20):
    """
    Goobjoog uses:
      Sports: https://goobjoog.com/qayb/cayaaraha/
      Health: https://goobjoog.com/qayb/saynis-tiknooloji/caafimaadka/
    Page 1 is the base; page 2+ is /page/{n}/
    """
    out = []
    base = base_url.rstrip("/") + "/"

    for page in range(1, max_pages + 1):
        if page == 1:
            url = base
        else:
            url = f"{base}page/{page}/"

        r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
        if r.status_code != 200:
            break

        soup = BeautifulSoup(r.text, "html.parser")
        links = soup.select("h2 a, h3 a")

        count = 0
        for a in links:
            title = a.get_text(strip=True)
            href = a.get("href", "")
            if not title or len(title) < 12:
                continue

            out.append({
                "text": title,
                "label": label,
                "source": "Goobjoog",
                "url": href
            })
    return out


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import xml.etree.ElementTree as ET

# Veri depolama listesi
rescue_data = []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# --- YÖNTEM 1: SOMALI WIKIPEDIA ---
# Wikipedia'da Kategori sayfaları: "Qeybta:..." diye başlar.
wiki_categories = [
    {"url": "https://so.wikipedia.org/wiki/Qeybta:Ciyaaraha", "label": "Sports"},
    {"url": "https://so.wikipedia.org/wiki/Qeybta:Caafimaadka", "label": "Health"},
    {"url": "https://so.wikipedia.org/wiki/Qeybta:Cudurada", "label": "Health"} # Hastalıklar
]

print("--- WIKIPEDIA OPERASYONU BAŞLADI ---")

for cat in wiki_categories:
    print(f"Wikipedia Kategori Taranıyor: {cat['label']}...")
    try:
        r = requests.get(cat['url'], headers=headers, timeout=10)
        soup = BeautifulSoup(r.content, 'html.parser')

        # Kategorideki sayfaların linklerini bul (mw-category-group içindedir)
        links = soup.select("#mw-pages a")

        count = 0
        for link in links:
            title = link.get_text().strip()
            # Wikipedia başlıklarını alıyoruz
            if len(title) > 5:
                rescue_data.append({
                    "text": title, # Wikipedia makale başlığı
                    "label": cat['label'],
                    "source": "Wikipedia"
                })
                count += 1
        print(f"✅ {count} adet Wikipedia başlığı alındı.")

    except Exception as e:
        print(f"Wikipedia Hatası: {e}")

# --- YÖNTEM 2: GOOGLE NEWS RSS (ASLA ENGELLENMEZ) ---
# Siteler seni engellese bile Google'ın beslemesinden başlıkları çekeriz.
rss_feeds = [
    # Google News Somali araması: Ciyaaraha (Spor)
    {"url": "https://news.google.com/rss/search?q=Ciyaaraha+Somalia&hl=so&gl=SO&ceid=SO:so", "label": "Sports"},
    # Google News Somali araması: Sport
    {"url": "https://news.google.com/rss/search?q=Sport+Somalia&hl=so&gl=SO&ceid=SO:so", "label": "Sports"},
    # Google News Somali araması: Caafimaadka (Sağlık)
    {"url": "https://news.google.com/rss/search?q=Caafimaadka+Somalia&hl=so&gl=SO&ceid=SO:so", "label": "Health"}
]

print("\n--- GOOGLE NEWS RSS OPERASYONU BAŞLADI ---")

for feed in rss_feeds:
    print(f"Google RSS Taranıyor: {feed['label']}...")
    try:
        r = requests.get(feed['url'], headers=headers, timeout=10)

        # XML yapısını ayrıştır
        root = ET.fromstring(r.content)

        count = 0
        # RSS içindeki 'item' etiketlerini gez
        for item in root.findall('.//item'):
            title = item.find('title').text

            # Başlık temizliği (Google bazen kaynak ismini sona ekler " - BBC", onu silelim)
            if "-" in title:
                title = title.split("-")[0].strip()

            if len(title) > 15:
                rescue_data.append({
                    "text": title,
                    "label": feed['label'],
                    "source": "Google News RSS"
                })
                count += 1

        print(f"✅ {count} adet Google News başlığı alındı.")

    except Exception as e:
        print(f"RSS Hatası: {e}")

# SONUÇLARI KAYDET
if len(rescue_data) > 0:
    df_wiki_google = pd.DataFrame(rescue_data)
    # Tekrarları sil
    df_wiki_google = df_wiki_google.drop_duplicates(subset=['text'])

    df_wiki_google.to_csv("somali_news_wiki_google.csv", index=False)
    print(f"\n🎉 TOPLAM {len(df_wiki_google)} ADET YENİ VERİ KURTARILDI!")
    print(df_wiki_google['label'].value_counts())
else:
    print("\n❌ Hiç veri bulunamadı. İnternet bağlantını kontrol et.")

--- WIKIPEDIA OPERASYONU BAŞLADI ---
Wikipedia Kategori Taranıyor: Sports...
✅ 0 adet Wikipedia başlığı alındı.
Wikipedia Kategori Taranıyor: Health...
✅ 0 adet Wikipedia başlığı alındı.
Wikipedia Kategori Taranıyor: Health...
✅ 0 adet Wikipedia başlığı alındı.

--- GOOGLE NEWS RSS OPERASYONU BAŞLADI ---
Google RSS Taranıyor: Sports...
✅ 66 adet Google News başlığı alındı.
Google RSS Taranıyor: Sports...
✅ 96 adet Google News başlığı alındı.
Google RSS Taranıyor: Health...
✅ 42 adet Google News başlığı alındı.

🎉 TOPLAM 202 ADET YENİ VERİ KURTARILDI!
label
Sports    160
Health     42
Name: count, dtype: int64


In [ ]:
import pandas as pd

# 1. Dosyaları Yükle
try:
    # Ana dosya (Politika/Dünya)
    df_main = pd.read_csv("somali_news_part1.csv")
    print(f"Ana Dosya: {len(df_main)} satır")
except:
    print("Ana dosya bulunamadı! Lütfen 'somali_news_part1.csv' ismini kontrol et.")
    df_main = pd.DataFrame() # Boş oluştur hata vermesin

try:
    # Google/Wiki dosyası
    df_new = pd.read_csv("somali_news_wiki_google.csv")
    print(f"Ek Dosya: {len(df_new)} satır")
except:
    df_new = pd.DataFrame()

# Varsa ara denemelerden kalanları da ekleyebilirsin, yoksa burayı geç
# df_part2 = pd.read_csv("somali_news_part2.csv") ... gibi

# 2. Hepsini Birleştir
df_final = pd.concat([df_main, df_new], ignore_index=True)

# 3. Temizlik
# Tekrar edenleri sil (Aynı haberi hem Google hem siteden çekmiş olabiliriz)
df_final = df_final.drop_duplicates(subset=['text'])

# Etiketleri standartlaştır (Büyük/Küçük harf hatası olmasın)
# Örneğin: "sports" -> "Sports"
df_final['label'] = df_final['label'].str.title().str.strip()

# 4. Veriyi Karıştır (Shuffle) - Çok önemli!
df_final = df_final.sample(frac=1).reset_index(drop=True)

# 5. Sonuç Raporu
print("\n  FİNAL VERİ SETİ RAPORU ---")
print(df_final['label'].value_counts())
print(f"\nTOPLAM SATIR SAYISI: {len(df_final)}")

# 6. Kaydet
df_final.to_csv("Somali_News_Classification_Dataset_Final.csv", index=False)
print("\n  DOSYA HAZIR! Hemen indirip Kaggle'a yükle.")

Ana dosya bulunamadı! Lütfen 'somali_news_part1.csv' ismini kontrol et.
Ek Dosya: 202 satır

--- 🏆 FİNAL VERİ SETİ RAPORU ---
label
Sports    160
Health     42
Name: count, dtype: int64

TOPLAM SATIR SAYISI: 202

✅ DOSYA HAZIR! Hemen indirip Kaggle'a yükle.


In [ ]:
import pandas as pd

# Load datasets
df1 = pd.read_csv("/content/somali_news_wiki_google.csv")
df2 = pd.read_csv("/content/Somali_News_Classification_Dataset_Final.csv")

# Combine datasets (row-wise)
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save new dataset
combined_df.to_csv("/content/Somali_News_Combined_Dataset.csv", index=False)

print("Combined dataset saved successfully!")
print("Total rows:", combined_df.shape[0])

FileNotFoundError: [Errno 2] No such file or directory: '/content/somali_news_wiki_google.csv'

In [ ]:
# İlk 4 satır
combined_df.head(4)


,text,label,source
0,Saddex Maamul oo qaadacay tartanka kubadda cag...,Sports,Google News RSS
1,Ururka Somalia Sisterhood: Kulankii Gabdhaha K...,Sports,Google News RSS
2,Sida ay halyeeyadan Afrika rajo ugu horseedaya...,Sports,Google News RSS
3,Ciyaartooy Caalami ah uu hogaaminayo Samuel Et...,Sports,Google News RSS


In [ ]:
# Toplam satır sayısı
len(combined_df)


404

In [ ]:
# Her sınıftan kaç tane var
combined_df['label'].value_counts()


,count
label,
Sports,320
Health,84


In [ ]:
len(combined_df)

404

In [ ]:
x = "/content/Somali_News_Classification_Dataset_Final.csv"

In [ ]:
len(x)

53

In [ ]:
import pandas as pd

# Load datasets
df1 = pd.read_csv("/content/somali_news_part1.csv")
df2 = pd.read_csv("/content/Somali_News_Combined_Dataset.csv")

# Combine
final_df = pd.concat([df1, df2], ignore_index=True)

# Save final dataset
final_df.to_csv("/content/Somali_News_Final_All.csv", index=False)

print("Final dataset created successfully!")
print("Total rows:", final_df.shape[0])

FileNotFoundError: [Errno 2] No such file or directory: '/content/Somali_News_Combined_Dataset.csv'

In [ ]:
final_df.head(4)


,text,label,source
0,BF oo maanta meel-mariyay miisaaniyadda qarank...,Politics,Caasimada
1,Faah-faahinta dil ka dhacay magaalada Gaalkacyo,Politics,Caasimada
2,Dowladda Mareykanka oo u yeertay safiirkeeda S...,Politics,Caasimada
3,Taiwan oo si adag ugu soo jawaabtay madaxweyne...,Politics,Caasimada


In [ ]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10914 entries, 0 to 10913
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10914 non-null  object
 1   label   10914 non-null  object
 2   source  10914 non-null  object
dtypes: object(3)
memory usage: 255.9+ KB


In [ ]:
# Her sınıftan kaç tane var
final_df['label'].value_counts()


,count
label,
Politics,5255
World,5255
Sports,320
Health,84


In [ ]:
import pandas as pd
import numpy as np

IN_PATH = "/content/Somali_News_Final_All.csv"
OUT_PATH = "/content/Somali_News_Balanced_5255.csv"

TARGET = 5255
LABEL_COL = "label"

# Metin sütunu adın "text" değilse aşağıda düzelt
TEXT_COL = "text"

df = pd.read_csv(IN_PATH)

# Temizlik: boşları at
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5]

print("Mevcut dağılım:\n", df[LABEL_COL].value_counts())

balanced_parts = []
rng = np.random.default_rng(42)

for label, g in df.groupby(LABEL_COL):
    g = g.copy()

    if len(g) >= TARGET:
        # 5255'ten fazlaysa aynı sayıya indir (sende Politics/World zaten 5255)
        g_bal = g.sample(n=TARGET, random_state=42)
    else:
        # Azsa: replacement ile oversample
        idx = rng.choice(g.index.to_numpy(), size=TARGET, replace=True)
        g_bal = g.loc[idx].copy()

    balanced_parts.append(g_bal)

df_bal = pd.concat(balanced_parts, ignore_index=True)

# Karıştır
df_bal = df_bal.sample(frac=1.0, random_state=42).reset_index(drop=True)

# İstersen tekrar eden başlıkları azaltmaya çalış (tamamen kaldırmak hedefi bozabilir)
# df_bal = df_bal.drop_duplicates(subset=[TEXT_COL, LABEL_COL]).reset_index(drop=True)

df_bal.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

print("\nYeni dağılım:\n", df_bal[LABEL_COL].value_counts())
print("Kaydedildi:", OUT_PATH)


Mevcut dağılım:
 label
Politics    5255
World       5255
Sports       320
Health        84
Name: count, dtype: int64

Yeni dağılım:
 label
Sports      5255
World       5255
Health      5255
Politics    5255
Name: count, dtype: int64
Kaydedildi: /content/Somali_News_Balanced_5255.csv


In [ ]:
df_health_top10 = df_bal[df_bal['label'] == 'Health'].head(10)
display(df_health_top10)

,text,label,source
2,Cimilada kulul oo la ogaaday in uu saameyso ca...,Health,Google News RSS
6,Caafimaadka Adduunka: Caalamka oo wajahaya xas...,Health,Google News RSS
13,Ra'isal wasaaraha dowlada Federaalka Somaliya ...,Health,Google News RSS
15,Soomaalida u safreysa Hindiya oo ka maarmaysa ...,Health,Google News RSS
18,Soomaalida u safreysa Hindiya oo ka maarmaysa ...,Health,Google News RSS
22,Daraasad: Huruudda ma waxay daweyn kartaa cudu...,Health,Google News RSS
26,Soomaalida u safreysa Hindiya oo ka maarmaysa ...,Health,Google News RSS
28,Adeegyada caafimaad oo aan laga hirgalin Somalia,Health,Google News RSS
29,"Wasaarada Caafimaadka oo ""laga afduubtay"" shaq...",Health,Google News RSS
36,XOG: Maxay Tahay Sababta Loo Xirey Taliyaha Ho...,Health,Google News RSS


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random

IN_PATH  = "/content/Somali_News_Final_All.csv"
OUT_PATH = "/content/Somali_News_Final_All_EXTENDED.csv"

TARGET_PER_LABEL = 5255
TEXT_COL = "text"
LABEL_COL = "label"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

def build_page_url(base, page):
    base = base.rstrip("/") + "/"
    if page == 1:
        return base
    return f"{base}page/{page}/"

def scrape_goobjoog_titles(base_url, label, need_count, existing_texts_norm, max_pages=2000, sleep_min=0.6, sleep_max=1.4):
    """
    Goobjoog kategori sayfalarından başlık+link çeker.
    need_count kadar YENİ (dupe olmayan) kayıt bulunca durur.
    """
    collected = []
    pages_hit = 0

    for page in range(1, max_pages + 1):
        url = build_page_url(base_url, page)

        try:
            r = requests.get(url, headers=HEADERS, timeout=25, allow_redirects=True)
            if r.status_code != 200:
                # kategori bitti / engel / hata
                # 404 görürsen genelde sayfa bitti demektir
                print(f"[{label}] Page {page}: HTTP {r.status_code} -> STOP")
                break

            soup = BeautifulSoup(r.text, "html.parser")

            # Goobjoog'ta çoğu zaman h2/h3 altında linkler var
            links = soup.select("h2 a, h3 a")

            if not links:
                print(f"[{label}] Page {page}: 0 link -> STOP")
                break

            new_in_page = 0
            for a in links:
                title = a.get_text(strip=True)
                href  = a.get("href", "").strip()

                if not title or len(title) < 12:
                    continue

                tnorm = " ".join(title.lower().split())
                if tnorm in existing_texts_norm:
                    continue

                existing_texts_norm.add(tnorm)
                collected.append({
                    "text": title,
                    "label": label,
                    "source": "Goobjoog",
                    "url": href
                })
                new_in_page += 1

                if len(collected) >= need_count:
                    print(f"[{label}] Reached target additions: {need_count}")
                    return collected

            pages_hit += 1
            print(f"[{label}] Page {page}: +{new_in_page} (total added {len(collected)}/{need_count})")

            # Sayfada hiç yeni veri çıkmıyorsa (hepsi dupe olabilir), birkaç sayfa sonra durdurmak mantıklı
            # ama burada direkt durdurmuyoruz; eski sayfalara geldikçe dupe artabilir.

            time.sleep(random.uniform(sleep_min, sleep_max))

        except requests.exceptions.RequestException as e:
            print(f"[{label}] Page {page}: request error -> {e} (continue)")
            continue

    return collected

# -------------------- MAIN --------------------
df = pd.read_csv(IN_PATH)

# temizlik
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)

print("Mevcut dağılım:")
print(df[LABEL_COL].value_counts())

# mevcut metinlerden normalize set
existing_texts_norm = set(" ".join(t.lower().split()) for t in df[TEXT_COL].tolist())

# hedef eksikler
counts = df[LABEL_COL].value_counts().to_dict()
need_sports = max(0, TARGET_PER_LABEL - counts.get("Sports", 0))
need_health = max(0, TARGET_PER_LABEL - counts.get("Health", 0))

print("\nEksikler:")
print("Sports need:", need_sports)
print("Health need:", need_health)

new_rows = []

# Sports (Goobjoog)
if need_sports > 0:
    sports_base = "https://goobjoog.com/qayb/cayaaraha/"
    new_rows += scrape_goobjoog_titles(
        base_url=sports_base,
        label="Sports",
        need_count=need_sports,
        existing_texts_norm=existing_texts_norm,
        max_pages=4000
    )

# Health (Goobjoog)
if need_health > 0:
    health_base = "https://goobjoog.com/qayb/saynis-tiknooloji/caafimaadka/"
    new_rows += scrape_goobjoog_titles(
        base_url=health_base,
        label="Health",
        need_count=need_health,
        existing_texts_norm=existing_texts_norm,
        max_pages=4000
    )

if not new_rows:
    print("\nYeni veri eklenemedi. (Erişim engeli / kategori bitti / çok fazla dupe olabilir.)")
else:
    df_new = pd.DataFrame(new_rows)
    df_out = pd.concat([df, df_new], ignore_index=True)

    print("\nYeni dağılım:")
    print(df_out[LABEL_COL].value_counts())

    df_out.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
    print("\nKaydedildi:", OUT_PATH)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Somali_News_Final_All.csv'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random

IN_PATH  = "/content/Somali_News_Final_All_EXTENDED.csv"
OUT_PATH = "/content/Somali_News_Final_All_EXTENDED_BALANCED.csv"

TARGET_PER_LABEL = 5255
TEXT_COL = "text"
LABEL_COL = "label"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

TIMEOUT = 25

def norm_text(s: str) -> str:
    return " ".join(str(s).lower().split())

def wp_page_url(base: str, page: int) -> str:
    base = base.rstrip("/") + "/"
    if page == 1:
        return base
    return f"{base}page/{page}/"

def scrape_titles(base_url, label, need_count, existing_norm_set,
                  selectors=("h2 a", "h3 a", "h4 a"),
                  max_pages=5000, sleep_min=0.7, sleep_max=1.6,
                  source_name="Site", url_mode="wp"):
    """
    Generic scraper:
    - url_mode="wp": base/page/{n}/
    - url_mode="fixed_page_param": base + str(n) (not used here)
    """
    out = []
    for page in range(1, max_pages + 1):
        if url_mode == "wp":
            url = wp_page_url(base_url, page)
        else:
            url = base_url + str(page)

        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[{source_name} - {label}] Page {page}: HTTP {r.status_code} -> STOP ({url})")
                break

            soup = BeautifulSoup(r.text, "html.parser")

            # collect links using selectors
            links = []
            for sel in selectors:
                links.extend(soup.select(sel))

            # remove duplicates by href+text quickly
            seen_local = set()
            cleaned = []
            for a in links:
                t = a.get_text(strip=True)
                h = (a.get("href") or "").strip()
                key = (t, h)
                if key in seen_local:
                    continue
                seen_local.add(key)
                cleaned.append((t, h))

            if not cleaned:
                print(f"[{source_name} - {label}] Page {page}: 0 items -> STOP")
                break

            added = 0
            for title, href in cleaned:
                if not title or len(title) < 12:
                    continue
                tn = norm_text(title)
                if tn in existing_norm_set:
                    continue

                existing_norm_set.add(tn)
                out.append({
                    "text": title,
                    "label": label,
                    "source": source_name,
                    "url": href
                })
                added += 1

                if len(out) >= need_count:
                    print(f"[{source_name} - {label}] Target reached: +{need_count}")
                    return out

            print(f"[{source_name} - {label}] Page {page}: +{added} (total +{len(out)}/{need_count})")

            # If the page adds nothing new repeatedly, it might be exhausted; we can stop early
            if added == 0 and page >= 5:
                # allow a couple more pages, but stop if it keeps happening
                # (simple heuristic)
                pass

            time.sleep(random.uniform(sleep_min, sleep_max))

        except requests.exceptions.RequestException as e:
            print(f"[{source_name} - {label}] Page {page}: request error -> {e} (continue)")
            continue

    return out

# -------------------- MAIN --------------------
df = pd.read_csv(IN_PATH)

# basic cleaning
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)

counts = df[LABEL_COL].value_counts()
print("Mevcut dağılım:\n", counts)

need_sports = max(0, TARGET_PER_LABEL - int(counts.get("Sports", 0)))
need_health = max(0, TARGET_PER_LABEL - int(counts.get("Health", 0)))
print("\nEksik:")
print("Sports need:", need_sports)
print("Health need:", need_health)

existing_norm = set(norm_text(t) for t in df[TEXT_COL].tolist())
new_rows = []

# 1) HEALTH: Radio Ergo Caafimaadka (çok sayfa)  :contentReference[oaicite:3]{index=3}
if need_health > 0:
    new_rows += scrape_titles(
        base_url="https://radioergo.org/category/caafimaadka/",
        label="Health",
        need_count=need_health,
        existing_norm_set=existing_norm,
        selectors=("h2 a", "h3 a", ".entry-title a", ".post-title a"),
        max_pages=8000,
        source_name="RadioErgo",
        url_mode="wp"
    )
    need_health -= sum(1 for r in new_rows if r["label"] == "Health")

# 2) HEALTH: Bosasoonline Caafimaadka  :contentReference[oaicite:4]{index=4}
if need_health > 0:
    add = scrape_titles(
        base_url="https://bosasoonline.com/category/arrimaha-bulshada/caafimaadka/",
        label="Health",
        need_count=need_health,
        existing_norm_set=existing_norm,
        selectors=("h2 a", "h3 a", ".entry-title a", ".post-title a"),
        max_pages=6000,
        source_name="Bosasoonline",
        url_mode="wp"
    )
    new_rows += add
    need_health -= len(add)

# 3) SPORTS: Horseed Media (tag ciyaaraha-puntland)  :contentReference[oaicite:5]{index=5}
if need_sports > 0:
    add = scrape_titles(
        base_url="https://horseedmedia.net/tag/ciyaaraha-puntland/",
        label="Sports",
        need_count=need_sports,
        existing_norm_set=existing_norm,
        selectors=("h1 a", "h2 a", "h3 a", ".entry-title a", ".post-title a"),
        max_pages=8000,
        source_name="HorseedMedia",
        url_mode="wp"
    )
    new_rows += add
    need_sports -= len(add)

# ---- Save ----
if not new_rows:
    print("\nYeni veri eklenemedi. (Erişim engeli / sayfa yapısı farklı olabilir.)")
else:
    df_new = pd.DataFrame(new_rows)

    # combine
    df_out = pd.concat([df, df_new], ignore_index=True)

    print("\nYeni dağılım:\n", df_out[LABEL_COL].value_counts())

    df_out.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
    print("\nKaydedildi:", OUT_PATH)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Somali_News_Final_All_EXTENDED.csv'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random

IN_PATH  = "/content/Somali_News_Final_All_EXTENDED_BALANCED.csv"
OUT_PATH = "/content/Somali_News_Final_All_EXTENDED_BALANCED_V2.csv"

TARGET = 5255
TEXT_COL = "text"
LABEL_COL = "label"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}
TIMEOUT = 25

# Somali health keywords (basit ama etkili)
HEALTH_KW = [
    "caafimaad", "caafimaadka", "cudur", "xanuun", "isbitaal", "dhakhtar",
    "tallaal", "daawo", "bukaan", "hooyo", "caruur", "nafaqo", "WHO",
    "covid", "faafa", "daryeel", "qalli", "kansar", "wadne", "maskax"
]

def norm(s: str) -> str:
    return " ".join(str(s).lower().split())

def wp_page(base: str, p: int) -> str:
    base = base.rstrip("/") + "/"
    return base if p == 1 else f"{base}page/{p}/"

def extract_titles_from_page(html: str):
    soup = BeautifulSoup(html, "html.parser")
    # WordPress sitelerde genelde bu sınıflar çalışır
    candidates = soup.select("h1 a, h2 a, h3 a, .entry-title a, .post-title a")
    items = []
    seen = set()
    for a in candidates:
        t = a.get_text(strip=True)
        u = (a.get("href") or "").strip()
        if not t or len(t) < 12:
            continue
        key = (t, u)
        if key in seen:
            continue
        seen.add(key)
        items.append((t, u))
    return items

def scrape_wp_category(base_url, label, need, existing_set, source, max_pages=10000, sleep=(0.6, 1.4)):
    out = []
    for p in range(1, max_pages + 1):
        if len(out) >= need:
            break
        url = wp_page(base_url, p)
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[{source} - {label}] Page {p}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles_from_page(r.text)
            if not items:
                print(f"[{source} - {label}] Page {p}: 0 items -> STOP")
                break

            added = 0
            for title, href in items:
                tn = norm(title)
                if tn in existing_set:
                    continue
                existing_set.add(tn)
                out.append({"text": title, "label": label, "source": source, "url": href})
                added += 1
                if len(out) >= need:
                    break

            print(f"[{source} - {label}] Page {p}: +{added} (total +{len(out)}/{need})")
            time.sleep(random.uniform(*sleep))
        except requests.exceptions.RequestException as e:
            print(f"[{source} - {label}] Page {p}: request error -> {e} (continue)")
            continue
    return out

def scrape_wp_category_health_filter(base_url, need, existing_set, source, max_pages=20000, sleep=(0.6, 1.4)):
    """
    Radiomuqdisho 'Wararka' gibi genel arşivden çekip Health keyword ile filtreler.
    """
    out = []
    for p in range(1, max_pages + 1):
        if len(out) >= need:
            break
        url = wp_page(base_url, p)
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[{source} - HealthFilter] Page {p}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles_from_page(r.text)
            if not items:
                print(f"[{source} - HealthFilter] Page {p}: 0 items -> STOP")
                break

            added = 0
            for title, href in items:
                tnl = norm(title)
                if tnl in existing_set:
                    continue
                # keyword filter
                if not any(k in tnl for k in HEALTH_KW):
                    continue

                existing_set.add(tnl)
                out.append({"text": title, "label": "Health", "source": source, "url": href})
                added += 1
                if len(out) >= need:
                    break

            print(f"[{source} - HealthFilter] Page {p}: +{added} (total +{len(out)}/{need})")
            # Eğer çok sayfa gezip hiç ekleme olmuyorsa aralık artırmak gerekebilir.
            time.sleep(random.uniform(*sleep))
        except requests.exceptions.RequestException as e:
            print(f"[{source} - HealthFilter] Page {p}: request error -> {e} (continue)")
            continue
    return out

# ---------------- MAIN ----------------
df = pd.read_csv(IN_PATH)
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)

counts = df[LABEL_COL].value_counts()
print("Mevcut dağılım:\n", counts)

need_sports = max(0, TARGET - int(counts.get("Sports", 0)))
need_health = max(0, TARGET - int(counts.get("Health", 0)))
print("\nEksik:")
print("Sports need:", need_sports)
print("Health need:", need_health)

existing_norm = set(norm(t) for t in df[TEXT_COL].tolist())
new_rows = []

# 1) SPORTS: Kooxda (çok güçlü)  :contentReference[oaicite:4]{index=4}
if need_sports > 0:
    new_rows += scrape_wp_category(
        base_url="https://kooxda.com/category/wararka-ciyaaraha-maanta/",
        label="Sports",
        need=need_sports,
        existing_set=existing_norm,
        source="Kooxda",
        max_pages=2000
    )

# Güncel ihtiyacı yeniden hesapla (ekledikten sonra)
added_sports = sum(1 for r in new_rows if r["label"] == "Sports")
need_sports2 = max(0, need_sports - added_sports)

# 2) SPORTS: Radio Muqdisho Ciyaaraha  :contentReference[oaicite:5]{index=5}
if need_sports2 > 0:
    add = scrape_wp_category(
        base_url="https://radiomuqdisho.so/category/ciyaaraha/",
        label="Sports",
        need=need_sports2,
        existing_set=existing_norm,
        source="RadioMuqdisho",
        max_pages=5000
    )
    new_rows += add

# 3) HEALTH: Radio Muqdisho Wararka (keyword filter ile)  :contentReference[oaicite:6]{index=6}
if need_health > 0:
    new_rows += scrape_wp_category_health_filter(
        base_url="https://radiomuqdisho.so/category/wararka/",
        need=need_health,
        existing_set=existing_norm,
        source="RadioMuqdisho",
        max_pages=3
    )

if not new_rows:
    print("\nYeni veri eklenemedi (erişim engeli / sayfa yapısı farklı olabilir).")
else:
    df_new = pd.DataFrame(new_rows)
    df_out = pd.concat([df, df_new], ignore_index=True)

    print("\nYeni dağılım:\n", df_out[LABEL_COL].value_counts())
    df_out.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
    print("\nKaydedildi:", OUT_PATH)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Somali_News_Final_All_EXTENDED_BALANCED.csv'

In [ ]:
import pandas as pd

try:
    df_part1 = pd.read_csv('/content/somali_news_part1.csv')
    print(f"Dataset '/content/somali_news_part1.csv' has {len(df_part1)} entries.")
    if 'label' in df_part1.columns:
        print(f"It contains {df_part1['label'].nunique()} unique labels.")
        print("Unique labels are:")
        print(df_part1['label'].unique())
    else:
        print("The 'label' column was not found in the dataset.")
except FileNotFoundError:
    print("Error: '/content/somali_news_part1.csv' not found. Please ensure the file exists.")
except Exception as e:
    print(f"An error occurred: {e}")

Dataset '/content/somali_news_part1.csv' has 10510 entries.
It contains 2 unique labels.
Unique labels are:
['Politics' 'World']


In [ ]:
# import pandas as pd

# PART1 = "/content/somali_news_part1.csv"      # Politics + World (10510)
# SPORTS = "/content/sports_5255.csv"           # Sports 5255 (senin hazır dosyan)
# ECONOMY = "/content/economy_5255.csv"         # Economy 5255 (toplayacağın dosya)
# OUT = "/content/Somali_News_Final_4Labels.csv"

# TEXT_COL = "text"
# LABEL_COL = "label"
# TARGET = 5255

# def clean_df(df):
#     df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
#     df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
#     df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
#     df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)
#     return df

# # 1) Politics & World
# df_pw = pd.read_csv(PART1)
# df_pw = clean_df(df_pw)
# df_pw = df_pw[df_pw[LABEL_COL].isin(["Politics", "World"])].copy()

# # 5255'e sabitle (fazlaysa kırp)
# df_pw_final = []
# for lab in ["Politics", "World"]:
#     g = df_pw[df_pw[LABEL_COL] == lab]
#     if len(g) < TARGET:
#         raise ValueError(f"{lab} için kayıt az: {len(g)} (hedef {TARGET})")
#     df_pw_final.append(g.sample(n=TARGET, random_state=42))
# df_pw = pd.concat(df_pw_final, ignore_index=True)

# # 2) Sports
# df_s = pd.read_csv(SPORTS)
# df_s = clean_df(df_s)
# df_s = df_s[df_s[LABEL_COL] == "Sports"].copy()
# if len(df_s) < TARGET:
#     raise ValueError(f"Sports için kayıt az: {len(df_s)} (hedef {TARGET})")
# df_s = df_s.sample(n=TARGET, random_state=42)

# # 3) Economy
# df_e = pd.read_csv(ECONOMY)
# df_e = clean_df(df_e)
# df_e = df_e[df_e[LABEL_COL] == "Economy"].copy()
# if len(df_e) < TARGET:
#     raise ValueError(f"Economy için kayıt az: {len(df_e)} (hedef {TARGET})")
# df_e = df_e.sample(n=TARGET, random_state=42)

# # 4) Merge + shuffle
# df_final = pd.concat([df_pw, df_s, df_e], ignore_index=True)
# df_final = df_final.sample(frac=1.0, random_state=42).reset_index(drop=True)

# print("Final dağılım:")
# print(df_final[LABEL_COL].value_counts())

# df_final.to_csv(OUT, index=False, encoding="utf-8-sig")
# print("Kaydedildi:", OUT)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random
import os

IN_PATH  = "/content/somali_news_part1.csv"
OUT_PATH = "/content/somali_news_part1_PLUS_SPORTS.csv"

TARGET_PER_LABEL = 5255
TEXT_COL = "text"
LABEL_COL = "label"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

def norm_text(s: str) -> str:
    return " ".join(str(s).lower().split())

def build_wp_page_url(base, page):
    base = base.rstrip("/") + "/"
    if page == 1:
        return base
    return f"{base}page/{page}/"

def extract_titles(html: str):
    soup = BeautifulSoup(html, "html.parser")
    # WP sitelerde en sık çalışan başlık seçicileri
    links = soup.select("h1 a, h2 a, h3 a, .entry-title a, .post-title a")
    items = []
    seen = set()
    for a in links:
        title = a.get_text(strip=True)
        href = (a.get("href") or "").strip()
        if not title or len(title) < 12:
            continue
        key = (title, href)
        if key in seen:
            continue
        seen.add(key)
        items.append((title, href))
    return items

def save_checkpoint(df, path):
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f">> CHECKPOINT SAVED: {path} (rows={len(df)})")

def scrape_category(base_url, label, need_count, existing_norm_set, source_name,
                    max_pages=10000, sleep_min=0.6, sleep_max=1.4,
                    checkpoint_every=200, df_base=None, out_path=None):
    """
    need_count kadar YENİ kayıt bulunca durur.
    df_base ve out_path verilirse, belirli aralıklarla ara kayıt alır.
    """
    collected = []
    for page in range(1, max_pages + 1):
        url = build_wp_page_url(base_url, page)

        try:
            r = requests.get(url, headers=HEADERS, timeout=25, allow_redirects=True)
            if r.status_code != 200:
                print(f"[{source_name} - {label}] Page {page}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles(r.text)
            if not items:
                print(f"[{source_name} - {label}] Page {page}: 0 items -> STOP ({url})")
                break

            added = 0
            for title, href in items:
                tn = norm_text(title)
                if tn in existing_norm_set:
                    continue
                existing_norm_set.add(tn)

                collected.append({
                    "text": title,
                    "label": label,
                    "source": source_name,
                    "url": href
                })
                added += 1

                # Checkpoint: her checkpoint_every kayıtta bir dosyayı güncelle
                if df_base is not None and out_path is not None and len(collected) % checkpoint_every == 0:
                    df_tmp = pd.concat([df_base, pd.DataFrame(collected)], ignore_index=True)
                    save_checkpoint(df_tmp, out_path)

                if len(collected) >= need_count:
                    print(f"[{source_name} - {label}] Target reached: +{need_count}")
                    return collected

            print(f"[{source_name} - {label}] Page {page}: +{added} (total +{len(collected)}/{need_count})")
            time.sleep(random.uniform(sleep_min, sleep_max))

        except requests.exceptions.RequestException as e:
            print(f"[{source_name} - {label}] Page {page}: request error -> {e} (continue)")
            continue

    return collected

# -------------------- MAIN --------------------
df = pd.read_csv(IN_PATH)

# temizlik
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)

print("Mevcut dağılım:")
print(df[LABEL_COL].value_counts())

# normalize set (dup önleme)
existing_norm = set(norm_text(t) for t in df[TEXT_COL].tolist())

counts = df[LABEL_COL].value_counts().to_dict()
need_sports = max(0, TARGET_PER_LABEL - counts.get("Sports", 0))

print("\nEksik:")
print("Sports need:", need_sports)

if need_sports == 0:
    print("Sports zaten 5255. Dosya aynen kaydediliyor.")
    df.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
    print("Kaydedildi:", OUT_PATH)
else:
    new_rows = []

    # 1) Kooxda (ana kaynak, çok büyük arşiv)
    new_rows += scrape_category(
        base_url="https://kooxda.com/category/wararka-ciyaaraha-maanta/",
        label="Sports",
        need_count=need_sports,
        existing_norm_set=existing_norm,
        source_name="Kooxda",
        max_pages=4000,
        df_base=df,
        out_path=OUT_PATH,
        checkpoint_every=200
    )

    # Eğer Kooxda yetmezse, Goobjoog yedek
    remain = need_sports - len(new_rows)
    if remain > 0:
        print(f"\nKooxda yetmedi, Goobjoog ile devam: kalan {remain}")
        new_rows += scrape_category(
            base_url="https://goobjoog.com/qayb/cayaaraha/",
            label="Sports",
            need_count=remain,
            existing_norm_set=existing_norm,
            source_name="Goobjoog",
            max_pages=6000,
            df_base=df,
            out_path=OUT_PATH,
            checkpoint_every=200
        )

    if not new_rows:
        print("\nYeni veri eklenemedi. (Erişim engeli / HTML değişmiş olabilir.)")
    else:
        df_new = pd.DataFrame(new_rows)
        df_out = pd.concat([df, df_new], ignore_index=True)

        print("\nYeni dağılım:")
        print(df_out[LABEL_COL].value_counts())

        # final kayıt
        save_checkpoint(df_out, OUT_PATH)


Mevcut dağılım:
label
Politics    5255
World       5255
Name: count, dtype: int64

Eksik:
Sports need: 5255
[Kooxda - Sports] Page 1: +47 (total +47/5255)
[Kooxda - Sports] Page 2: +18 (total +65/5255)
[Kooxda - Sports] Page 3: +18 (total +83/5255)
[Kooxda - Sports] Page 4: +19 (total +102/5255)
[Kooxda - Sports] Page 5: +19 (total +121/5255)
[Kooxda - Sports] Page 6: +19 (total +140/5255)
[Kooxda - Sports] Page 7: +19 (total +159/5255)
[Kooxda - Sports] Page 8: +19 (total +178/5255)
[Kooxda - Sports] Page 9: +20 (total +198/5255)
>> CHECKPOINT SAVED: /content/somali_news_part1_PLUS_SPORTS.csv (rows=10710)
[Kooxda - Sports] Page 10: +20 (total +218/5255)
[Kooxda - Sports] Page 11: +19 (total +237/5255)
[Kooxda - Sports] Page 12: +18 (total +255/5255)
[Kooxda - Sports] Page 13: +20 (total +275/5255)
[Kooxda - Sports] Page 14: +20 (total +295/5255)
[Kooxda - Sports] Page 15: +19 (total +314/5255)
[Kooxda - Sports] Page 16: +20 (total +334/5255)
[Kooxda - Sports] Page 17: +19 (total +353/

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random

IN_PATH  = "/content/somali_news_part1_PLUS_SPORTS.csv"
OUT_PATH = "/content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY.csv"

TARGET = 5255
TEXT_COL = "text"
LABEL_COL = "label"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}
TIMEOUT = 25

# Economy keywords (genişletildi)
ECON_KW = [
    # Somali
    "dhaqaale", "dhaqaalaha", "dhaqaaleed", "ganacsi", "ganacsiga", "ganacsato",
    "sicir", "sicirbarar", "sicir-barar", "qiime", "qiimaha", "kharash", "miisaaniyad",
    "dakhli", "khasaaro", "faa'iido", "faaido", "deyn", "dayn", "maalgashi", "maalgelin",
    "shirkad", "shirkadaha", "saamiyo", "saamiga", "suq", "suuq", "suqyada", "suuqyada",
    "deked", "dhoof", "dhoofin", "soo dejin", "soodejin", "soodajin", "xawaalad",
    "lacag", "shilin", "doolar", "sarif", "sarrif", "bangiga", "bangiyada", "bank", "banks",
    "shaqo", "shaqo abuur", "mushahar", "cashuur", "canshuur", "canshuuraha",
    "hawlgal", "maalqabeen", "maalqabeenka", "ganacsiga gudaha", "ganacsiga caalamiga",
    # English (Somali sites often mix)
    "business", "economy", "economic", "finance", "financial", "investment", "market",
    "import", "export", "trade", "tax", "budget", "revenue", "profit", "loss",
]

def norm(s: str) -> str:
    return " ".join(str(s).lower().split())

def wp_page(base: str, p: int) -> str:
    base = base.rstrip("/") + "/"
    return base if p == 1 else f"{base}page/{p}/"

def extract_titles(html: str):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("h1 a, h2 a, h3 a, .entry-title a, .post-title a")
    out, seen = [], set()
    for a in links:
        t = a.get_text(strip=True)
        u = (a.get("href") or "").strip()
        if not t or len(t) < 12:
            continue
        key = (t, u)
        if key in seen:
            continue
        seen.add(key)
        out.append((t, u))
    return out

def save_checkpoint(df, path):
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f">> CHECKPOINT SAVED: {path} (rows={len(df)})")

def scrape_wp_category(base_url, label, need, existing_set, source,
                       max_pages=10000, sleep=(0.6, 1.4),
                       checkpoint_every=200, df_base=None, out_path=None,
                       stop_after_zero_pages=10):
    """
    Kategori sayfasından direkt başlık çeker.
    Art arda stop_after_zero_pages sayfa 0 eklerse durur.
    """
    collected = []
    zero_streak = 0

    for p in range(1, max_pages + 1):
        if len(collected) >= need:
            break

        url = wp_page(base_url, p)
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[{source} - {label}] Page {p}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles(r.text)
            if not items:
                print(f"[{source} - {label}] Page {p}: 0 items -> STOP ({url})")
                break

            added = 0
            for title, href in items:
                tn = norm(title)
                if tn in existing_set:
                    continue
                existing_set.add(tn)
                collected.append({"text": title, "label": label, "source": source, "url": href})
                added += 1

                if df_base is not None and out_path is not None and len(collected) % checkpoint_every == 0:
                    df_tmp = pd.concat([df_base, pd.DataFrame(collected)], ignore_index=True)
                    save_checkpoint(df_tmp, out_path)

                if len(collected) >= need:
                    break

            if added == 0:
                zero_streak += 1
            else:
                zero_streak = 0

            print(f"[{source} - {label}] Page {p}: +{added} (total +{len(collected)}/{need})")

            if zero_streak >= stop_after_zero_pages:
                print(f"[{source} - {label}] {zero_streak} sayfa üst üste 0 ekledi -> STOP")
                break

            time.sleep(random.uniform(*sleep))

        except requests.exceptions.RequestException as e:
            print(f"[{source} - {label}] Page {p}: request error -> {e} (continue)")
            continue

    return collected

def scrape_wp_with_keyword_filter(base_url, need, existing_set, source,
                                 max_pages=20000, sleep=(0.6, 1.4),
                                 checkpoint_every=200, df_base=None, out_path=None,
                                 stop_after_zero_pages=25):
    """
    Genel arşivden başlık çeker, ECON_KW ile filtreleyerek Economy üretir.
    Art arda stop_after_zero_pages sayfa 0 eklerse durur.
    """
    collected = []
    zero_streak = 0

    for p in range(1, max_pages + 1):
        if len(collected) >= need:
            break

        url = wp_page(base_url, p)
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[{source} - EconFilter] Page {p}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles(r.text)
            if not items:
                print(f"[{source} - EconFilter] Page {p}: 0 items -> STOP ({url})")
                break

            added = 0
            for title, href in items:
                tnl = norm(title)
                if tnl in existing_set:
                    continue
                if not any(k in tnl for k in ECON_KW):
                    continue

                existing_set.add(tnl)
                collected.append({"text": title, "label": "Economy", "source": source, "url": href})
                added += 1

                if df_base is not None and out_path is not None and len(collected) % checkpoint_every == 0:
                    df_tmp = pd.concat([df_base, pd.DataFrame(collected)], ignore_index=True)
                    save_checkpoint(df_tmp, out_path)

                if len(collected) >= need:
                    break

            if added == 0:
                zero_streak += 1
            else:
                zero_streak = 0

            print(f"[{source} - EconFilter] Page {p}: +{added} (total +{len(collected)}/{need})")

            if zero_streak >= stop_after_zero_pages:
                print(f"[{source} - EconFilter] {zero_streak} sayfa üst üste 0 ekledi -> STOP")
                break

            time.sleep(random.uniform(*sleep))

        except requests.exceptions.RequestException as e:
            print(f"[{source} - EconFilter] Page {p}: request error -> {e} (continue)")
            continue

    return collected

# ---------------- MAIN ----------------
df = pd.read_csv(IN_PATH)
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)

print("Mevcut dağılım:\n", df[LABEL_COL].value_counts())

existing_norm = set(norm(t) for t in df[TEXT_COL].tolist())
counts = df[LABEL_COL].value_counts().to_dict()
need_econ = max(0, TARGET - counts.get("Economy", 0))

print("\nEksik:")
print("Economy need:", need_econ)

if need_econ == 0:
    df.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
    print("Economy zaten var. Kaydedildi:", OUT_PATH)
else:
    new_rows = []

    # 1) Kooxda (Economy / Business) — çok güçlü kaynak; eğer sayfa yapısı uygunsa hızlı doldurur
    # Kooxda'da bu kategori adı değişebilir. Eğer 404 alırsan sorun değil, diğerlerine geçer.
    new_rows += scrape_wp_category(
        base_url="https://kooxda.com/category/ganacsi/",
        label="Economy",
        need=need_econ,
        existing_set=existing_norm,
        source="Kooxda",
        max_pages=8000,
        df_base=df,
        out_path=OUT_PATH,
        checkpoint_every=200,
        stop_after_zero_pages=10
    )

    remain = need_econ - len(new_rows)
    if remain > 0:
        print(f"\nKooxda sonrası kalan Economy: {remain}")

        # 2) Goobjoog - Dhaqaale
        add = scrape_wp_category(
            base_url="https://goobjoog.com/qayb/dhaqaale/",
            label="Economy",
            need=remain,
            existing_set=existing_norm,
            source="Goobjoog",
            max_pages=8000,
            df_base=df,
            out_path=OUT_PATH,
            checkpoint_every=200,
            stop_after_zero_pages=10
        )
        new_rows += add
        remain = need_econ - len(new_rows)

    if remain > 0:
        print(f"\nKategori kaynakları yetmedi. Keyword filtre ile tamamlanıyor: kalan {remain}")

        # 3) Radio Muqdisho - Wararka (keyword filter)
        add = scrape_wp_with_keyword_filter(
            base_url="https://radiomuqdisho.so/category/wararka/",
            need=remain,
            existing_set=existing_norm,
            source="RadioMuqdisho",
            max_pages=20000,
            df_base=df,
            out_path=OUT_PATH,
            checkpoint_every=200,
            stop_after_zero_pages=25
        )
        new_rows += add

    if not new_rows:
        print("\nEconomy için yeni veri eklenemedi (erişim/HTML).")
    else:
        df_new = pd.DataFrame(new_rows)
        df_out = pd.concat([df, df_new], ignore_index=True)

        print("\nYeni dağılım:\n", df_out[LABEL_COL].value_counts())
        save_checkpoint(df_out, OUT_PATH)


Mevcut dağılım:
 label
Politics    5255
World       5255
Sports      5255
Name: count, dtype: int64

Eksik:
Economy need: 5255
[Kooxda - Economy] Page 1: HTTP 404 -> STOP (https://kooxda.com/category/ganacsi/)

Kooxda sonrası kalan Economy: 5255
[Goobjoog - Economy] Page 1: HTTP 404 -> STOP (https://goobjoog.com/qayb/dhaqaale/)

Kategori kaynakları yetmedi. Keyword filtre ile tamamlanıyor: kalan 5255
[RadioMuqdisho - EconFilter] Page 1: +1 (total +1/5255)
[RadioMuqdisho - EconFilter] Page 2: +0 (total +1/5255)
[RadioMuqdisho - EconFilter] Page 3: +0 (total +1/5255)
[RadioMuqdisho - EconFilter] Page 4: +3 (total +4/5255)
[RadioMuqdisho - EconFilter] Page 5: +0 (total +4/5255)
[RadioMuqdisho - EconFilter] Page 6: +3 (total +7/5255)
[RadioMuqdisho - EconFilter] Page 7: +0 (total +7/5255)
[RadioMuqdisho - EconFilter] Page 8: +0 (total +7/5255)
[RadioMuqdisho - EconFilter] Page 9: +0 (total +7/5255)
[RadioMuqdisho - EconFilter] Page 10: +3 (total +10/5255)
[RadioMuqdisho - EconFilter] Page 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random

IN_PATH  = "/content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY.csv"
OUT_PATH = "/content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_WIKI.csv"

HEADERS = {"User-Agent": "Mozilla/5.0"}
TEXT_COL="text"; LABEL_COL="label"

WIKI_CATS = [
    "https://so.wikipedia.org/wiki/Category:Dhaqaale",
    "https://so.wikipedia.org/wiki/Category:Ganacsi",
    "https://so.wikipedia.org/wiki/Category:Dhaqaalaha_Soomaaliya"
]

def norm(s):
    return " ".join(str(s).lower().split())

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=25)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def list_category_pages(cat_url, max_pages=500):
    soup = get_soup(cat_url)
    pages = []
    # category members list
    for a in soup.select("#mw-pages a"):
        href = a.get("href","")
        title = a.get_text(strip=True)

        # "Bogga xiga" gibi navigasyon linklerini atla
        if "Bogga" in title or "xiga" in title or "hore" in title:
            continue

        if href.startswith("/wiki/") and not href.startswith("/wiki/Category:"):
            # Help/Template/Special gibi isim alanlarını atla
            if href.startswith("/wiki/Help:") or href.startswith("/wiki/Special:") or href.startswith("/wiki/Template:"):
                continue
            pages.append(("https://so.wikipedia.org"+href, title))
    return pages[:max_pages]

def fetch_title_and_intro(page_url):
    soup = get_soup(page_url)

    h1 = soup.select_one("h1")
    if not h1:
        return None

    title = h1.get_text(strip=True)

    # farklı alanlardan ilk anlamlı paragrafı bul
    candidates = soup.select("div.mw-parser-output > p") + soup.select("#mw-content-text p")

    intro = ""
    for p in candidates:
        txt = p.get_text(" ", strip=True)
        # çok kısa stub’lar için eşiği düşürdük
        if txt and len(txt) >= 15:
            intro = txt
            break

    if not intro:
        return None

    text = f"{title}. {intro}"
    return text

# load dataset
df = pd.read_csv(IN_PATH)
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()

existing = set(norm(x) for x in df[TEXT_COL].tolist())

new_rows = []
for cat in WIKI_CATS:
    pages = list_category_pages(cat)
    print(f"[WIKI] {cat} -> {len(pages)} page")

    for url, _ in pages:
        try:
            text = fetch_title_and_intro(url)
            if not text:
                continue

            tn = norm(text)
            if tn in existing:
                continue

            existing.add(tn)
            new_rows.append({
                "text": text,
                "label": "Economy",
                "source": "SomaliWikipedia",
                "url": url
            })

            if len(new_rows) % 10 == 0:
                df_tmp = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
                df_tmp.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
                print(">> CHECKPOINT SAVED", OUT_PATH, "added", len(new_rows))

            time.sleep(random.uniform(0.4, 1.0))

        except Exception:
            continue

if new_rows:
    df_out = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    print("Yeni dağılım:\n", df_out[LABEL_COL].value_counts())
    df_out.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
    print("Kaydedildi:", OUT_PATH, "added:", len(new_rows))
else:
    print("Wikipedia'dan yeni veri eklenemedi (intro bulunamadı veya hepsi dupe).")


[WIKI] https://so.wikipedia.org/wiki/Category:Dhaqaale -> 8 page
[WIKI] https://so.wikipedia.org/wiki/Category:Ganacsi -> 26 page
>> CHECKPOINT SAVED /content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_WIKI.csv added 10
>> CHECKPOINT SAVED /content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_WIKI.csv added 20
>> CHECKPOINT SAVED /content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_WIKI.csv added 30
[WIKI] https://so.wikipedia.org/wiki/Category:Dhaqaalaha_Soomaaliya -> 4 page
Yeni dağılım:
 label
Politics    5255
World       5255
Sports      5255
Economy      506
Name: count, dtype: int64
Kaydedildi: /content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_WIKI.csv added: 38


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random

IN_PATH  = "/content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_WIKI.csv"
OUT_PATH = "/content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_FULL.csv"

TARGET = 5255
TEXT_COL = "text"
LABEL_COL = "label"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}
TIMEOUT = 25

# Economy keyword filter (Somali + sık geçen ekonomi terimleri)
ECON_KW = [
    "dhaqaale", "dhaqaalaha", "ganacsi", "ganacsiga", "sicir", "sicir-barar",
    "lacag", "shilin", "doolar", "bangiga", "bank", "suq", "suuq",
    "shirkad", "shirkadaha", "maalgashi", "maalgelin", "cashuur", "canshuur",
    "deked", "dhoof", "soo dejin", "soodejin", "import", "export",
    "shaqo", "shaqo abuur", "qiime", "qiimaha", "kharash", "miisaaniyad",
    "dakhli", "khasaaro", "faa'iido", "faaido", "deyn", "dayn",
    "warshad", "warshadaha", "ganacsato", "raasamaal", "suqgeyn"
]

def norm(s: str) -> str:
    return " ".join(str(s).lower().split())

def wp_page(base: str, p: int) -> str:
    base = base.rstrip("/") + "/"
    return base if p == 1 else f"{base}page/{p}/"

def extract_titles(html: str):
    soup = BeautifulSoup(html, "html.parser")
    # geniş seçiciler: WP temaları farklı olabilir
    links = soup.select("h1 a, h2 a, h3 a, .entry-title a, .post-title a, article h2 a, article h3 a")
    out = []
    seen = set()
    for a in links:
        t = a.get_text(" ", strip=True)
        u = (a.get("href") or "").strip()
        if not t or len(t) < 12:
            continue
        key = (t, u)
        if key in seen:
            continue
        seen.add(key)
        out.append((t, u))
    return out

def save_checkpoint(df, path):
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f">> CHECKPOINT SAVED: {path} (rows={len(df)})")

def scrape_wp_with_keyword_filter(base_url, need, existing_set, source,
                                 max_pages=200000, sleep=(0.5, 1.2),
                                 checkpoint_every=300, df_base=None, out_path=None):
    """
    Genel arşivden başlık çekip ECON_KW ile filtreleyerek Economy üretir.
    """
    collected = []
    no_add_pages = 0

    for p in range(1, max_pages + 1):
        if len(collected) >= need:
            break

        url = wp_page(base_url, p)
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[{source}] Page {p}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles(r.text)
            if not items:
                print(f"[{source}] Page {p}: 0 items -> STOP ({url})")
                break

            added = 0
            for title, href in items:
                tnl = norm(title)
                if tnl in existing_set:
                    continue
                if not any(k in tnl for k in ECON_KW):
                    continue

                existing_set.add(tnl)
                collected.append({"text": title, "label": "Economy", "source": source, "url": href})
                added += 1

                if df_base is not None and out_path is not None and len(collected) % checkpoint_every == 0:
                    df_tmp = pd.concat([df_base, pd.DataFrame(collected)], ignore_index=True)
                    save_checkpoint(df_tmp, out_path)

                if len(collected) >= need:
                    break

            print(f"[{source}] Page {p}: +{added} (total +{len(collected)}/{need})")

            if added == 0:
                no_add_pages += 1
            else:
                no_add_pages = 0

            # çok uzun süre hiç eklemiyorsa (tamamen dupe/uyumsuz), dur
            if no_add_pages >= 80:
                print(f"[{source}] 80 sayfadır yeni Economy çıkmıyor -> STOP")
                break

            time.sleep(random.uniform(*sleep))

        except requests.exceptions.RequestException as e:
            print(f"[{source}] Page {p}: request error -> {e} (continue)")
            continue

    return collected

# ---------------- MAIN ----------------
df = pd.read_csv(IN_PATH)
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)

print("Mevcut dağılım:\n", df[LABEL_COL].value_counts())

existing_norm = set(norm(t) for t in df[TEXT_COL].tolist())
counts = df[LABEL_COL].value_counts().to_dict()
need_econ = max(0, TARGET - counts.get("Economy", 0))

print("\nEksik:")
print("Economy need:", need_econ)

if need_econ == 0:
    save_checkpoint(df, OUT_PATH)
else:
    new_rows = []

    # 1) SomaliChamber News (çok sayfalı)
    add1 = scrape_wp_with_keyword_filter(
        base_url="https://somalichamber.so/category/news/",
        need=need_econ,
        existing_set=existing_norm,
        source="SomaliChamber",
        max_pages=20000,
        df_base=df,
        out_path=OUT_PATH,
        checkpoint_every=300
    )
    new_rows += add1

    remain = need_econ - len(new_rows)
    if remain > 0:
        print(f"\nSomaliChamber yetmedi. Kalan Economy: {remain}")

        # 2) Puntland MOF (çok geniş arşiv)
        add2 = scrape_wp_with_keyword_filter(
            base_url="https://mof.pl.so/category/warka/",
            need=remain,
            existing_set=existing_norm,
            source="PuntlandMOF",
            max_pages=200000,
            df_base=df,
            out_path=OUT_PATH,
            checkpoint_every=300
        )
        new_rows += add2

    remain = need_econ - len(new_rows)
    if remain > 0:
        print(f"\nPuntlandMOF yetmedi. Kalan Economy: {remain}")

        # 3) Radio Waamo (genel haber ama arşiv büyük)
        add3 = scrape_wp_with_keyword_filter(
            base_url="https://radiowaamo.so/category/wararka/",
            need=remain,
            existing_set=existing_norm,
            source="RadioWaamo",
            max_pages=200000,
            df_base=df,
            out_path=OUT_PATH,
            checkpoint_every=300
        )
        new_rows += add3

    if not new_rows:
        print("\nEconomy için yeni veri eklenemedi. (Erişim/HTML değişmiş olabilir.)")
    else:
        df_new = pd.DataFrame(new_rows)
        df_out = pd.concat([df, df_new], ignore_index=True)

        print("\nYeni dağılım:\n", df_out[LABEL_COL].value_counts())
        save_checkpoint(df_out, OUT_PATH)


Mevcut dağılım:
 label
Politics    5255
World       5255
Sports      5255
Economy      506
Name: count, dtype: int64

Eksik:
Economy need: 4749
[SomaliChamber] Page 1: +3 (total +3/4749)
[SomaliChamber] Page 2: +5 (total +8/4749)
[SomaliChamber] Page 3: +3 (total +11/4749)
[SomaliChamber] Page 4: +6 (total +17/4749)
[SomaliChamber] Page 5: +6 (total +23/4749)
[SomaliChamber] Page 6: +5 (total +28/4749)
[SomaliChamber] Page 7: +8 (total +36/4749)
[SomaliChamber] Page 8: +2 (total +38/4749)
[SomaliChamber] Page 9: +6 (total +44/4749)
[SomaliChamber] Page 10: +8 (total +52/4749)
[SomaliChamber] Page 11: +4 (total +56/4749)
[SomaliChamber] Page 12: +5 (total +61/4749)
[SomaliChamber] Page 13: +3 (total +64/4749)
[SomaliChamber] Page 14: HTTP 404 -> STOP (https://somalichamber.so/category/news/page/14/)

SomaliChamber yetmedi. Kalan Economy: 4685
[PuntlandMOF] Page 1: +6 (total +6/4685)
[PuntlandMOF] Page 2: +5 (total +11/4685)
[PuntlandMOF] Page 3: +5 (total +16/4685)
[PuntlandMOF] Page 4:

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, random

# ---- INPUT / OUTPUT ----
IN_PATH  = "/content/somali_news_part1_PLUS_SPORTS_PLUS_ECONOMY_FULL.csv"
OUT_PATH = "/content/somali_news_PART1_FINAL_PW.csv"

TARGET = 5255
TEXT_COL = "text"
LABEL_COL = "label"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}
TIMEOUT = 25

# Somali Economy / Business keyword set (genişletilmiş)
ECON_KW = [
    "dhaqaale","dhaqaalaha","ganacsi","ganacsiga","maalgashi","maalgelin","miisaaniyad","dakhli","kharash",
    "sicir","sicir-barar","qiime","qiimaha","suuq","suq","shirkad","shirkadaha","bangiga","bank","lacag",
    "shilin","doolar","cashuur","canshuur","deked","dhoof","soodejin","soo dejin","import","export",
    "shaqo","shaqo abuur","faa'iido","faaido","khasaaro","deyn","dayn","xawaalad","xawaaladaha",
    "dhoofinta","soodejinta","ganacsade","ganacsatada","warshad","warshadaha","saami","saamiyada",
    "suqgeyn","suuqgeyn","shidaal","batrool","gaas","koronto","biil","canshuuraha"
]

def norm(s: str) -> str:
    return " ".join(str(s).lower().split())

def extract_titles_generic(html: str):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("h1 a, h2 a, h3 a, h4 a, .entry-title a, .post-title a")
    out = []
    seen = set()
    for a in links:
        t = a.get_text(strip=True)
        u = (a.get("href") or "").strip()
        if not t or len(t) < 12:
            continue
        key = (t, u)
        if key in seen:
            continue
        seen.add(key)
        out.append((t, u))
    return out

def save_checkpoint(df, path, added_so_far=None):
    df.to_csv(path, index=False, encoding="utf-8-sig")
    msg = f">> CHECKPOINT SAVED: {path} (rows={len(df)})"
    if added_so_far is not None:
        msg += f" | economy_added={added_so_far}"
    print(msg)

def scrape_hiiraan_wararkamaanta_econ(need, existing_set, df_base, checkpoint_every=200, sleep=(0.4, 1.0), max_pages=999999):
    """
    Hiiraan: https://www.hiiraan.com/wararkamaanta.php?page=N
    Başlıklar sayfada <h3> gibi görünüyor, genel selector ile çekiyoruz.
    Economy keyword filtre uygulanır.
    """
    base = "https://www.hiiraan.com/wararkamaanta.php?page="
    collected = []
    for p in range(1, max_pages + 1):
        if len(collected) >= need:
            break

        url = f"{base}{p}"
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[Hiiraan] Page {p}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles_generic(r.text)
            if not items:
                print(f"[Hiiraan] Page {p}: 0 items -> STOP ({url})")
                break

            added = 0
            for title, href in items:
                tnl = norm(title)
                if tnl in existing_set:
                    continue
                if not any(k in tnl for k in ECON_KW):
                    continue

                existing_set.add(tnl)
                collected.append({"text": title, "label": "Economy", "source": "Hiiraan", "url": href})
                added += 1

                if len(collected) % checkpoint_every == 0:
                    df_tmp = pd.concat([df_base, pd.DataFrame(collected)], ignore_index=True)
                    save_checkpoint(df_tmp, OUT_PATH, added_so_far=len(collected))

                if len(collected) >= need:
                    break

            print(f"[Hiiraan] Page {p}: +{added} (total +{len(collected)}/{need})")
            time.sleep(random.uniform(*sleep))

        except requests.exceptions.RequestException as e:
            print(f"[Hiiraan] Page {p}: request error -> {e} (continue)")
            continue

    return collected

def wp_page(base: str, p: int) -> str:
    base = base.rstrip("/") + "/"
    return base if p == 1 else f"{base}page/{p}/"

def scrape_somalilandtoday_news_econ(need, existing_set, df_base, checkpoint_every=200, sleep=(0.5, 1.2), max_pages=50000):
    """
    Somalilandtoday: https://somalilandtoday.com/category/news/page/N/
    Economy keyword filtre uygulanır.
    """
    base = "https://somalilandtoday.com/category/news/"
    collected = []
    for p in range(1, max_pages + 1):
        if len(collected) >= need:
            break

        url = wp_page(base, p)
        try:
            r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, allow_redirects=True)
            if r.status_code != 200:
                print(f"[Somalilandtoday] Page {p}: HTTP {r.status_code} -> STOP ({url})")
                break

            items = extract_titles_generic(r.text)
            if not items:
                print(f"[Somalilandtoday] Page {p}: 0 items -> STOP ({url})")
                break

            added = 0
            for title, href in items:
                tnl = norm(title)
                if tnl in existing_set:
                    continue
                if not any(k in tnl for k in ECON_KW):
                    continue

                existing_set.add(tnl)
                collected.append({"text": title, "label": "Economy", "source": "Somalilandtoday", "url": href})
                added += 1

                if len(collected) % checkpoint_every == 0:
                    df_tmp = pd.concat([df_base, pd.DataFrame(collected)], ignore_index=True)
                    save_checkpoint(df_tmp, OUT_PATH, added_so_far=len(collected))

                if len(collected) >= need:
                    break

            print(f"[Somalilandtoday] Page {p}: +{added} (total +{len(collected)}/{need})")
            time.sleep(random.uniform(*sleep))

        except requests.exceptions.RequestException as e:
            print(f"[Somalilandtoday] Page {p}: request error -> {e} (continue)")
            continue

    return collected

# ---------------- MAIN ----------------
df = pd.read_csv(IN_PATH)
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df = df[df[TEXT_COL].str.len() > 5].reset_index(drop=True)

print("Mevcut dağılım:")
print(df[LABEL_COL].value_counts())

existing_norm = set(norm(t) for t in df[TEXT_COL].tolist())
counts = df[LABEL_COL].value_counts().to_dict()
need_econ = max(0, TARGET - counts.get("Economy", 0))
print("\nEconomy need:", need_econ)

if need_econ == 0:
    save_checkpoint(df, OUT_PATH, added_so_far=0)
else:
    new_rows = []

    # 1) Hiiraan (en büyük arşiv)  -> önce buradan doldur
    add1 = scrape_hiiraan_wararkamaanta_econ(
        need=need_econ,
        existing_set=existing_norm,
        df_base=df,
        checkpoint_every=200
    )
    new_rows += add1

    remain = need_econ - len(add1)
    if remain > 0:
        print(f"\nHiiraan yetmedi. Kalan Economy: {remain}")

        # 2) Somalilandtoday -> kalan kadar tamamla
        add2 = scrape_somalilandtoday_news_econ(
            need=remain,
            existing_set=existing_norm,
            df_base=df,
            checkpoint_every=200
        )
        new_rows += add2

    if not new_rows:
        print("\nEconomy için yeni veri eklenemedi (erişim/HTML/dupe/keyword).")
        save_checkpoint(df, OUT_PATH, added_so_far=0)
    else:
        df_new = pd.DataFrame(new_rows)
        df_out = pd.concat([df, df_new], ignore_index=True)

        print("\nYeni dağılım:")
        print(df_out[LABEL_COL].value_counts())

        save_checkpoint(df_out, OUT_PATH, added_so_far=len(df_new))


Mevcut dağılım:
label
Politics    5255
World       5255
Sports      5255
Economy     1701
Name: count, dtype: int64

Economy need: 3554
[Hiiraan] Page 1: +3 (total +3/3554)
[Hiiraan] Page 2: +0 (total +3/3554)
[Hiiraan] Page 3: +1 (total +4/3554)
[Hiiraan] Page 4: +2 (total +6/3554)
[Hiiraan] Page 5: +1 (total +7/3554)
[Hiiraan] Page 6: +2 (total +9/3554)
[Hiiraan] Page 7: +1 (total +10/3554)
[Hiiraan] Page 8: +1 (total +11/3554)
[Hiiraan] Page 9: +2 (total +13/3554)
[Hiiraan] Page 10: +1 (total +14/3554)
[Hiiraan] Page 11: +2 (total +16/3554)
[Hiiraan] Page 12: +1 (total +17/3554)
[Hiiraan] Page 13: +0 (total +17/3554)
[Hiiraan] Page 14: +1 (total +18/3554)
[Hiiraan] Page 15: +1 (total +19/3554)
[Hiiraan] Page 16: +0 (total +19/3554)
[Hiiraan] Page 17: +0 (total +19/3554)
[Hiiraan] Page 18: +1 (total +20/3554)
[Hiiraan] Page 19: +1 (total +21/3554)
[Hiiraan] Page 20: +2 (total +23/3554)
[Hiiraan] Page 21: +0 (total +23/3554)
[Hiiraan] Page 22: +2 (total +25/3554)
[Hiiraan] Page 23: +2